In [4]:
import tensorflow_addons as tfa

import numpy as np
import tensorflow as tf
import pickle
from tensorflow.keras import layers , activations , models , preprocessing, utils

In [5]:
#load data and tag

reviews = []
responses = []

with open('review.dat', 'rb') as f:
    rev = pickle.load(f)
with open('reply.dat', 'rb') as f:
    rep = pickle.load(f)
    
for i in range(len(rep)):
    responses.append( '<START> ' + rep[i] + ' <END>' )
    
reviews = rev

In [6]:
reviews = reviews[0:int(len(reviews)/2)]
responses = responses[0:int(len(responses)/2)]

In [9]:
#tokenize
tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts( reviews + responses )

VOCAB_SIZE = len( tokenizer.word_index )+1
#print( 'VOCAB SIZE : {}'.format( VOCAB_SIZE ))

vocab = []
for word in tokenizer.word_index:
    vocab.append( word )

def tokenize( sentences ):
    tokens_list = []
    vocabulary = []
    for sentence in sentences:
        sentence = sentence.lower()
        sentence = re.sub( '[^a-zA-Z]', ' ', sentence )
        tokens = sentence.split()
        vocabulary += tokens
        tokens_list.append( tokens )
    return tokens_list , vocabulary

In [10]:
# encoder_input_data
tokenized_reviews = tokenizer.texts_to_sequences( reviews )
maxlen_reviews = max( [ len(x) for x in tokenized_reviews ] )
padded_reviews = preprocessing.sequence.pad_sequences( tokenized_reviews , maxlen=maxlen_reviews , padding='post' )
encoder_input_data = np.array( padded_reviews )
print( encoder_input_data.shape , maxlen_reviews )

# decoder_input_data
tokenized_responses = tokenizer.texts_to_sequences( responses )
maxlen_responses = max( [ len(x) for x in tokenized_responses ] )
padded_responses = preprocessing.sequence.pad_sequences( tokenized_responses , maxlen=maxlen_responses , padding='post' )
decoder_input_data = np.array( padded_responses )
print( decoder_input_data.shape , maxlen_responses )

# decoder_output_data
tokenized_responses = tokenizer.texts_to_sequences( responses )
for i in range(len(tokenized_responses)) :
    tokenized_responses[i] = tokenized_responses[i][1:]
padded_responses = preprocessing.sequence.pad_sequences( tokenized_responses , maxlen=maxlen_responses , padding='post' )
onehot_responses = utils.to_categorical( padded_responses , VOCAB_SIZE )
decoder_output_data = np.array( onehot_responses )
print( decoder_output_data.shape )

(536, 961) 961
(536, 809) 809
(536, 809, 8656)


In [11]:
encoder_inputs = tf.keras.layers.Input(shape=( maxlen_reviews , ))
encoder_embedding = tf.keras.layers.Embedding( VOCAB_SIZE, 200 , mask_zero=True ) (encoder_inputs)
encoder_outputs , state_h , state_c = tf.keras.layers.LSTM( 200 , return_state=True )( encoder_embedding )
encoder_states = [ state_h , state_c ]

decoder_inputs = tf.keras.layers.Input(shape=( maxlen_responses ,  ))
decoder_embedding = tf.keras.layers.Embedding( VOCAB_SIZE, 200 , mask_zero=True) (decoder_inputs)
decoder_lstm = tf.keras.layers.LSTM( 200 , return_state=True , return_sequences=True )
decoder_outputs , _ , _ = decoder_lstm ( decoder_embedding , initial_state=encoder_states )
decoder_dense = tf.keras.layers.Dense( VOCAB_SIZE , activation=tf.keras.activations.softmax ) 
output = decoder_dense ( decoder_outputs )

model = tf.keras.models.Model([encoder_inputs, decoder_inputs], output )
model.compile(optimizer=tf.keras.optimizers.RMSprop(), loss='categorical_crossentropy')

model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 961)]        0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 809)]        0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 961, 200)     1731200     input_1[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 809, 200)     1731200     input_2[0][0]                    
______________________________________________________________________________________________

In [ ]:
#attempt larger batch size if comp can handle it (50)
model.fit([encoder_input_data , decoder_input_data], decoder_output_data, batch_size=25, epochs=150 ) 
model.save( 'model.h5' ) 

Epoch 1/150
22/22 [==============================] - 278s 12s/step - loss: 0.7850
Epoch 2/150
22/22 [==============================] - 285s 13s/step - loss: 0.7080
Epoch 3/150
22/22 [==============================] - 304s 14s/step - loss: 0.6995
Epoch 4/150
22/22 [==============================] - 312s 14s/step - loss: 0.6901
Epoch 5/150
22/22 [==============================] - 312s 14s/step - loss: 0.6836
Epoch 6/150
22/22 [==============================] - 315s 14s/step - loss: 0.6773
Epoch 7/150
22/22 [==============================] - 308s 14s/step - loss: 0.6709
Epoch 8/150
22/22 [==============================] - 305s 14s/step - loss: 0.6636
Epoch 9/150
22/22 [==============================] - 313s 14s/step - loss: 0.6569
Epoch 10/150
22/22 [==============================] - 333s 15s/step - loss: 0.6579
Epoch 11/150
22/22 [==============================] - 345s 16s/step - loss: 0.6514
Epoch 12/150
22/22 [==============================] - 329s 15s/step - loss: 0.6417
Epoch 13/150
